# Income Prediction Model - Data Science Pipeline

This notebook demonstrates the complete data science pipeline for the Income Prediction project. We'll go through the following steps:

1. Data Loading and Exploration
2. Data Cleaning and Preprocessing
3. Feature Engineering
4. Feature Selection
5. Model Training and Evaluation
6. Model Saving for Production Use

The goal is to predict whether a person's income exceeds $50K per year based on census data.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.feature_selection import SelectKBest, f_classif
import pickle

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

# Set random seed for reproducibility
np.random.seed(42)

## 1. Data Loading and Initial Exploration

In [ ]:
# Define column names since the dataset doesn't include them
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num', 
    'marital_status', 'occupation', 'relationship', 'race', 'sex', 
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income'
]

# Load the dataset
df = pd.read_csv('adult.data', names=column_names, sep=', ', engine='python')
print(f"Dataset shape: {df.shape}")
df.head()

In [ ]:
# Check data types and missing values
df.info()

In [ ]:
# Get descriptive statistics
df.describe()

In [ ]:
# Check the distribution of the target variable
income_counts = df['income'].value_counts()
print(income_counts)

# Visualize the distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='income', data=df)
plt.title('Income Distribution')
plt.ylabel('Count')
plt.show()

# Calculate the percentage
print(f"Percentage of >50K: {income_counts[' >50K'] / len(df) * 100:.2f}%")
print(f"Percentage of <=50K: {income_counts[' <=50K'] / len(df) * 100:.2f}%")

## 2. Data Cleaning

In [ ]:
# Check for missing values (in this dataset, missing values are marked as '?')
print("Missing values (marked as '?'):")
for column in df.columns:
    missing_count = (df[column] == ' ?').sum()
    if missing_count > 0:
        print(f"{column}: {missing_count}")

In [ ]:
# Replace '?' with NaN and then handle missing values
for column in df.columns:
    df[column] = df[column].replace(' ?', np.nan)

# Handle missing values - for simplicity, we'll drop rows with missing values
# In a real project, you might want to use imputation instead
df_cleaned = df.dropna()
print(f"Shape after removing missing values: {df_cleaned.shape}")

In [ ]:
# Check for duplicates
duplicates = df_cleaned.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
if duplicates > 0:
    df_cleaned = df_cleaned.drop_duplicates()
    print("Duplicates removed.")

In [ ]:
# Strip leading/trailing whitespace from string columns
for column in df_cleaned.select_dtypes(include=['object']).columns:
    df_cleaned[column] = df_cleaned[column].str.strip()

## 3. Exploratory Data Analysis

In [ ]:
# Age distribution
plt.figure(figsize=(10, 6))
sns.histplot(data=df_cleaned, x='age', hue='income', bins=30, kde=True)
plt.title('Age Distribution by Income')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

In [ ]:
# Education level and income
plt.figure(figsize=(12, 6))
education_order = df_cleaned.groupby('education')['education_num'].mean().sort_values().index
sns.countplot(data=df_cleaned, y='education', hue='income', order=education_order)
plt.title('Education Level vs Income')
plt.xlabel('Count')
plt.ylabel('Education Level')
plt.show()

In [ ]:
# Hours per week distribution
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_cleaned, x='income', y='hours_per_week')
plt.title('Hours per Week by Income')
plt.xlabel('Income')
plt.ylabel('Hours per Week')
plt.show()

In [ ]:
# Occupation and income
plt.figure(figsize=(12, 8))
occupation_income = df_cleaned.groupby('occupation')['income'].apply(lambda x: (x == '>50K').mean()).sort_values()
sns.barplot(x=occupation_income.values, y=occupation_income.index)
plt.title('Percentage of >50K Income by Occupation')
plt.xlabel('Percentage with >50K Income')
plt.ylabel('Occupation')
plt.show()

In [ ]:
# Correlation between numerical features
numerical_cols = ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
plt.figure(figsize=(10, 8))
sns.heatmap(df_cleaned[numerical_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Numerical Features')
plt.show()

## 4. Feature Engineering

In [ ]:
# Create age groups
df_cleaned['age_group'] = pd.cut(
    df_cleaned['age'], 
    bins=[0, 25, 35, 45, 55, 65, 100], 
    labels=['<25', '25-35', '35-45', '45-55', '55-65', '65+']
)

# Create hours worked category
df_cleaned['work_intensity'] = pd.cut(
    df_cleaned['hours_per_week'], 
    bins=[0, 20, 40, 60, 100], 
    labels=['Part-time', 'Full-time', 'Overtime', 'Workaholic']
)

# Create a feature for education level (simplified)
education_map = {
    'Preschool': 'Low',
    '1st-4th': 'Low',
    '5th-6th': 'Low',
    '7th-8th': 'Low',
    '9th': 'Low',
    '10th': 'Medium',
    '11th': 'Medium',
    '12th': 'Medium',
    'HS-grad': 'Medium',
    'Some-college': 'Medium',
    'Assoc-voc': 'High',
    'Assoc-acdm': 'High',
    'Bachelors': 'High',
    'Masters': 'Very High',
    'Prof-school': 'Very High',
    'Doctorate': 'Very High'
}
df_cleaned['education_level'] = df_cleaned['education'].map(education_map)

# Check the new features
df_cleaned[['age', 'age_group', 'hours_per_week', 'work_intensity', 'education', 'education_level']].head()

## 5. Target Preparation

In [ ]:
# Convert the target to binary (1 for >50K, 0 for <=50K)
df_cleaned['income'] = df_cleaned['income'].apply(lambda x: 1 if x == '>50K' else 0)
print(f"Target variable distribution:\n{df_cleaned['income'].value_counts()}")

## 6. Feature Selection

In [ ]:
# Identify categorical and numerical columns
categorical_cols = df_cleaned.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df_cleaned.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remove target from features
if 'income' in numerical_cols:
    numerical_cols.remove('income')

# Select K best features
X = df_cleaned.drop(columns=['income'])
y = df_cleaned['income']

# Using SelectKBest for numerical features
selector = SelectKBest(f_classif, k=min(10, len(numerical_cols)))
selector.fit(df_cleaned[numerical_cols], y)
selected_numerical = [numerical_cols[i] for i in selector.get_support(indices=True)]
print(f"Selected numerical features: {selected_numerical}")

# For categorical features, we'll select based on domain knowledge
selected_categorical = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']
print(f"Selected categorical features: {selected_categorical}")

selected_features = selected_numerical + selected_categorical
print(f"Total selected features: {len(selected_features)}")

## 7. Data Processing and Model Training

In [ ]:
# Split the data
X = df_cleaned[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

In [ ]:
# Create preprocessing pipeline
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, selected_numerical),
        ('cat', categorical_transformer, selected_categorical)
    ])

In [ ]:
# Define models to compare
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
}

best_accuracy = 0
best_model = None
best_model_name = None
results = {}

for name, model in models.items():
    # Create pipeline with preprocessing and model
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Train the model
    print(f"Training {name}...")
    pipeline.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))
    
    # Save results
    results[name] = {
        'accuracy': accuracy,
        'pipeline': pipeline,
        'predictions': y_pred
    }
    
    # Save the best model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = pipeline
        best_model_name = name

print(f"\nBest model: {best_model_name} with accuracy: {best_accuracy:.4f}")

## 8. Model Evaluation

In [ ]:
# Confusion Matrix for the best model
y_pred = results[best_model_name]['predictions']
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['<=50K', '>50K'],
            yticklabels=['<=50K', '>50K'])
plt.title(f'Confusion Matrix - {best_model_name}')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# ROC Curve for all models
plt.figure(figsize=(10, 8))

for name, result in results.items():
    pipeline = result['pipeline']
    y_proba = pipeline.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Different Models')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Feature importance for Random Forest
if 'Random Forest' in results:
    rf_pipeline = results['Random Forest']['pipeline']
    rf_model = rf_pipeline.named_steps['model']
    
    # Get feature names after preprocessing
    preprocessor = rf_pipeline.named_steps['preprocessor']
    cat_features = preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names_out(selected_categorical)
    feature_names = np.concatenate([selected_numerical, cat_features])
    
    # Get feature importances
    importances = rf_model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    # Plot top 20 features
    plt.figure(figsize=(12, 8))
    plt.title('Feature Importances - Random Forest')
    plt.bar(range(min(20, len(importances))), importances[indices[:20]], align='center')
    plt.xticks(range(min(20, len(importances))), feature_names[indices[:20]], rotation=90)
    plt.tight_layout()
    plt.show()

## 9. Save Model for Production

In [ ]:
# Save the best model
if best_accuracy >= 0.8:  # Check if accuracy meets requirement
    # Save the model using pickle
    with open('income_prediction_model.pkl', 'wb') as file:
        pickle.dump(best_model, file)
    print("Model saved successfully with accuracy above 80%!")
    
    # Also save the feature list for future use
    with open('selected_features.pkl', 'wb') as file:
        pickle.dump(selected_features, file)
    print("Selected features saved.")
    
    # Save the preprocessor for future use
    with open('preprocessor.pkl', 'wb') as file:
        pickle.dump(preprocessor, file)
    print("Preprocessor saved.")
else:
    print(f"Model accuracy ({best_accuracy:.4f}) is below 80%. Consider improving the model.")

## 10. Test Predictions

In [ ]:
# Make predictions on a few test samples
sample_indices = np.random.choice(len(X_test), min(5, len(X_test)), replace=False)
samples = X_test.iloc[sample_indices]
true_labels = y_test.iloc[sample_indices]
predictions = best_model.predict(samples)
probabilities = best_model.predict_proba(samples)[:, 1]

print("Sample predictions:")
for i, (idx, sample) in enumerate(samples.iterrows()):
    print(f"Sample {i+1}:")
    for feature, value in sample.items():
        print(f"  {feature}: {value}")
    print(f"  True label: {'Income >50K' if true_labels.iloc[i] == 1 else 'Income <=50K'}")
    print(f"  Predicted label: {'Income >50K' if predictions[i] == 1 else 'Income <=50K'}")
    print(f"  Confidence: {probabilities[i]*100:.2f}%")
    print()

## Conclusion

In this notebook, we've built a complete data science pipeline for income prediction:

1. We loaded and explored the Census Income dataset
2. We cleaned the data by handling missing values and duplicates
3. We performed feature engineering to create new informative features
4. We selected the most relevant features for our model
5. We trained and compared multiple machine learning models
6. We evaluated the models using various metrics
7. We saved the best model for use in our Django application

The best model achieved good accuracy and can now be used to predict whether a person's income exceeds $50K based on their demographic and employment information.